Python Training using PyTorch framework TempCNN model on FRH01-FRH03

In [ ]:
import os
import time
import torch
import pandas as pd
import sklearn.metrics
import breizhcrops
from torch.utils.data import DataLoader, ConcatDataset
from torch.optim import Adam
from tqdm import tqdm 


from breizhcrops import BreizhCrops
from breizhcrops.models import TempCNN

In [ ]:
# ---- Configuration ----
DATA_PATH = "hdf5-files/breizh_data"                  
LEVEL = "L1C"
PRELOAD_RAM = False
BATCH_SIZE = 1024
EPOCHS = 11
LEARNING_RATE = 2.38e-4           
WEIGHT_DECAY = 5.18e-5              
NUM_WORKERS = 4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LOGDIR = "/to save the training logs/log_tempcnn"
MODEL_NAME = "tempcnn_py_model"


# ---- Load Data ----
print("Loading BreizhCrops datasets")
frh01 = BreizhCrops(region="frh01", root=DATA_PATH, level=LEVEL, preload_ram=PRELOAD_RAM)
frh02 = BreizhCrops(region="frh02", root=DATA_PATH, level=LEVEL, preload_ram=PRELOAD_RAM)
frh03 = BreizhCrops(region="frh03", root=DATA_PATH, level=LEVEL, preload_ram=PRELOAD_RAM)

train_dataset = ConcatDataset([frh01, frh02, frh03])
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=torch.cuda.is_available()  
)


num_classes = len(frh01.classes)
ndims = 13 if LEVEL == "L1C" else 10
sequencelength = 45
class_names = frh01.classes
print(f"Total samples: {len(train_dataset)} | Batches/epoch: {len(train_loader)}")

In [ ]:
# ---- Initialize Model ----
model = TempCNN(input_dim=ndims, num_classes=num_classes, sequencelength=sequencelength).to(DEVICE)
optimizer = Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
criterion = torch.nn.CrossEntropyLoss()

os.makedirs(LOGDIR, exist_ok=True)
log = []

In [ ]:
# ---- Training Loop ----
for epoch in range(EPOCHS):
    model.train()
    start_time = time.time()
    losses = []
    y_true_all, y_pred_all = [], []

    for x, y_true, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        x, y_true = x.to(DEVICE), y_true.to(DEVICE)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y_true)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        y_true_all.append(y_true.cpu())
        y_pred_all.append(y_pred.argmax(dim=1).cpu())

    duration = time.time() - start_time
    iters_per_sec = len(train_loader) / duration

    y_true_all = torch.cat(y_true_all)
    y_pred_all = torch.cat(y_pred_all)

 # ---- Metrics ----
    metrics = {
        "epoch": epoch,
        "loss": sum(losses) / len(losses),
        "accuracy": sklearn.metrics.accuracy_score(y_true_all, y_pred_all),
        "kappa": sklearn.metrics.cohen_kappa_score(y_true_all, y_pred_all),
        "f1_micro": sklearn.metrics.f1_score(y_true_all, y_pred_all, average="micro"),
        "f1_macro": sklearn.metrics.f1_score(y_true_all, y_pred_all, average="macro"),
        "f1_weighted": sklearn.metrics.f1_score(y_true_all, y_pred_all, average="weighted"),
        "recall_micro": sklearn.metrics.recall_score(y_true_all, y_pred_all, average="micro"),
        "recall_macro": sklearn.metrics.recall_score(y_true_all, y_pred_all, average="macro"),
        "recall_weighted": sklearn.metrics.recall_score(y_true_all, y_pred_all, average="weighted"),
        "precision_micro": sklearn.metrics.precision_score(y_true_all, y_pred_all, average="micro"),
        "precision_macro": sklearn.metrics.precision_score(y_true_all, y_pred_all, average="macro"),
        "precision_weighted": sklearn.metrics.precision_score(y_true_all, y_pred_all, average="weighted"),
        "iters_per_sec": iters_per_sec,
        "epoch_time_sec": duration
    }

    print(f"[Epoch {epoch+1}] Loss={metrics['loss']:.4f}, Acc={metrics['accuracy']:.4f}, "
          f"F1_macro={metrics['f1_macro']:.4f}, Kappa={metrics['kappa']:.4f}, "
          f"{iters_per_sec:.2f} it/s, Time={duration:.1f}s")

    
    # ---- Save overall metrics ----
    log.append(metrics)
    pd.DataFrame(log).to_csv(os.path.join(LOGDIR, f"{MODEL_NAME}_log.csv"), index=False)

    # ---- Save per-class metrics ----
    class_report = sklearn.metrics.classification_report(y_true_all, y_pred_all,
                                                         target_names=class_names, output_dict=True)
    df_class_report = pd.DataFrame(class_report).transpose()
    df_class_report.to_csv(os.path.join(LOGDIR, f"{MODEL_NAME}_epoch{epoch+1}_per_class.csv"))

    # ---- Confusion matrix ----
    cm = sklearn.metrics.confusion_matrix(y_true_all, y_pred_all)
    pd.DataFrame(cm, index=class_names, columns=class_names).to_csv(
        os.path.join(LOGDIR, f"{MODEL_NAME}_epoch{epoch+1}_confusion.csv"))

In [ ]:
# ---- Save final model ----
torch.save(model.state_dict(), os.path.join(LOGDIR, f"{MODEL_NAME}.pt"))
print(f"Model saved to {LOGDIR}/{MODEL_NAME}.pt")

Exporting the trained model to ONNX model

In [ ]:
# ---- Export to ONNX ----
dummy_input = torch.randn(1, sequencelength, ndims).to(DEVICE)
onnx_path = os.path.join(LOGDIR, f"{MODEL_NAME}.onnx")

torch.onnx.export(
    model, dummy_input, onnx_path,
    input_names=['input'], output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}},
    export_params=True, opset_version=11
)

print(f"Model exported to ONNX: {onnx_path}")

To validate the ONNX model using ONNX checker

In [ ]:
# ---- ONNX checker ----
import onnx
from onnx import checker

def is_onnx_model_valid(path):
    try:
        model = onnx.load(path)
        checker.check_model(model)
        print("ONNX model is valid!")
        return True
    except onnx.checker.ValidationError as e:
        print("ONNX model is not valid!")
        print(f"Reason: {e}")
        return False

is_onnx_model_valid("log_tempcnn/tempcnn_py_model.onnx")

Evaluation on the FRH04 Region of the trained PyTorch model 

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
frh04 = BreizhCrops(region="frh04", root=DATA_PATH, level=LEVEL, preload_ram=PRELOAD_RAM)
model.load_state_dict(torch.load("log_tempcnn/tempcnn_py_model.pt",map_location=device))
model.to(device)
model.eval()

In [ ]:
x, y_true, field_id = frh04[4005]  
x = x.unsqueeze(0).to(device)
with torch.no_grad():
    logits = model(x)  
    probs = torch.nn.functional.softmax(logits, dim=1)
    predicted_class = torch.argmax(probs, dim=1).item()
print(f"Field ID: {field_id}")
print(f"True Class Index: {y_true} → {frh04.classname[y_true]}")
print(f"Predicted Class Index: {predicted_class} → {frh04.classname[predicted_class]}")

In [ ]:
from sklearn.metrics import classification_report
from collections import defaultdict
import torch
import numpy as np


y_true_all = []
y_pred_all = []
conf_per_class = defaultdict(list)

for i in range(len(frh04)):
    x, y_true, field_id = frh04[i]
    x = x.unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(x)
        probs = torch.nn.functional.softmax(logits, dim=1)
        y_pred = torch.argmax(probs, dim=1).item()
        confidence = torch.max(probs).item()


    y_true_all.append(y_true)
    y_pred_all.append(y_pred)
    conf_per_class[y_pred].append(confidence)

# Classification report
target_names = frh04.classname
print(" Python Classification Report - PyTorch : ")
print(classification_report(y_true_all, y_pred_all, target_names=target_names,zero_division=0))

# Average confidence per predicted class
print("\n Python Average Confidence per Predicted Class - PyTorch: ")
for class_idx, confidences in conf_per_class.items():
    avg_conf = np.mean(confidences)
    print(f"{class_idx:>2} ({frh04.classname[class_idx]:<15}): {avg_conf:.4f}")


PyTorch ONNX Evaluation on the FRH04 Region 

In [ ]:
import onnxruntime as ort
import numpy as np
from sklearn.metrics import classification_report
from collections import defaultdict

sess_options = ort.SessionOptions()
sess_options.intra_op_num_threads = 1
sess_options.inter_op_num_threads = 1
ort.set_default_logger_severity(2)

session = ort.InferenceSession(
    "log_tempcnn/tempcnn_py_model.onnx",
    sess_options=sess_options,
    providers=providers
)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

y_true_all = []
y_pred_all = []
conf_per_class = defaultdict(list)

for i in range(len(frh04)):
    x, y_true, _ = frh04[i]
    x_np = x.unsqueeze(0).numpy().astype(np.float32)  
    log_probs = session.run([output_name], {input_name: x_np})[0][0] 
    probs = np.exp(log_probs)
    y_pred = int(np.argmax(probs))
    confidence = float(np.max(probs))
    y_true_all.append(y_true)
    y_pred_all.append(y_pred)
    conf_per_class[y_pred].append(confidence)

# Classification Report
print("\n Python Classification Report - ONNX:")
print(classification_report(
    y_true_all,
    y_pred_all,
    target_names=frh04.classname,
    zero_division=0
))

#  Average Confidence per Predicted Class
print("\n Python Average Confidence per Predicted Class - ONNX:")
for class_idx, confs in conf_per_class.items():
    avg_conf = np.mean(confs)
    class_name = frh04.classname[class_idx]
    print(f"{class_idx:>2} ({class_name:<20}): {avg_conf:.4f}")



Flux ONNX Evaluation on the FRH04 Region 

In [ ]:
import onnxruntime as ort
import numpy as np
from sklearn.metrics import classification_report
from collections import defaultdict

sess_options = ort.SessionOptions()
sess_options.intra_op_num_threads = 1
sess_options.inter_op_num_threads = 1
ort.set_default_logger_severity(2)

providers = (
    ["CUDAExecutionProvider", "CPUExecutionProvider"]
    if "CUDAExecutionProvider" in ort.get_available_providers()
    else ["CPUExecutionProvider"]
)
# Load Julia ONNX model
session = ort.InferenceSession(
    "notebooks/tempcnn_julia_batch.onnx",
    sess_options=sess_options,
    providers=providers
)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

def softmax(x):
    x = x - np.max(x, axis=-1, keepdims=True)  
    e = np.exp(x)
    return e / np.sum(e, axis=-1, keepdims=True)
y_true_all = []
y_pred_all = []
conf_per_class = defaultdict(list)

for i in range(len(frh04)):
    x, y_true, _ = frh04[i]
    x_np = x.unsqueeze(0).numpy().astype(np.float32)  
    x_np = np.transpose(x_np, (0, 2, 1)).copy() 
    logits = session.run([output_name], {input_name: x_np})[0][0]  
    probs = softmax(logits)
    y_pred = int(np.argmax(probs))
    confidence = float(np.max(probs))
    y_true_all.append(y_true)
    y_pred_all.append(y_pred)
    conf_per_class[y_pred].append(confidence)

#  Classification Report
print("\n Julia Classification Report - ONNX:")
print(classification_report(
    y_true_all,
    y_pred_all,
    target_names=frh04.classname,
    zero_division=0
))

#  Average Confidence per Predicted Class
print("\n Julia Average Confidence per Predicted Class - ONNX:")
for class_idx, confs in conf_per_class.items():
    avg_conf = np.mean(confs)
    class_name = frh04.classname[class_idx]
    print(f"{class_idx:>2} ({class_name:<20}): {avg_conf:.4f}")
